<a href="https://colab.research.google.com/github/sum-coderepo/DeepLearning-Pytorch/blob/master/PytorchTutorials/Ful_conNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model.forward(x).shape)

torch.Size([64, 10])


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 1

In [12]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [16]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [17]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [18]:
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network

In [19]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train Network

In [22]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

In [23]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [24]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56030 / 60000 with accuracy 93.38
Checking accuracy on test data
Got 9340 / 10000 with accuracy 93.40


# Predicting Insurance price

In [200]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [202]:
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [203]:
train_data = pd.read_csv("dataset/Concrete_Data_Yeh.csv")
train_data.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [266]:
Q1=train_data.quantile(0.25)
Q3=train_data.quantile(0.75)

IQR=Q3-Q1
IQR

cement              157.625
slag                142.950
flyash              118.300
water                27.100
superplasticizer     10.200
coarseaggregate      97.400
fineaggregate        93.050
age                  49.000
csMPa                22.425
dtype: float64

In [267]:
train_data= train_data[~((train_data < (Q1 - 1.5 * IQR)) |(train_data > (Q3 + 1.5 * IQR))).any(axis=1)]

In [169]:
#train_data = pd.read_csv("dataset/Concrete_Data_Yeh.csv")
#train_data = pd.get_dummies(train_data, columns=["sex", "smoker", "region"])
#cols = ["age", "bmi", "children",  "sex_female", "sex_male", "smoker_no", "smoker_yes","region_northeast",  "region_northwest", "region_southeast", "region_southwest", "charges"]
#train_data = train_data.reindex(columns=cols)
#train_data.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,charges
0,19,27.900,0,1,0,0,1,0,0,0,1,16884.92400
1,18,33.770,1,0,1,1,0,0,0,1,0,1725.55230
2,28,33.000,3,0,1,1,0,0,0,1,0,4449.46200
3,33,22.705,0,0,1,1,0,0,1,0,0,21984.47061
4,32,28.880,0,0,1,1,0,0,1,0,0,3866.85520


In [268]:
scaler = MinMaxScaler()

In [269]:
X_train = train_data.iloc[:,0:-1].to_numpy()
X_train = scaler.fit_transform(X_train)

In [ ]:
scaler.

In [270]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(train_data.iloc[:,-1].to_numpy()).unsqueeze(1)

In [271]:
X_train

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.7384, 0.2336, 0.2269],
        [0.3744, 0.3332, 0.0000,  ..., 0.3808, 0.2165, 0.7479],
        [0.6347, 0.2777, 0.0000,  ..., 0.3808, 0.0000, 0.2269],
        ...,
        [0.1062, 0.4075, 0.5427,  ..., 0.2657, 0.5299, 0.2269],
        [0.1304, 0.5457, 0.0000,  ..., 0.5483, 0.5553, 0.2269],
        [0.3628, 0.2938, 0.3913,  ..., 0.1846, 0.4772, 0.2269]],
       dtype=torch.float64)

In [272]:
X_train.shape, y_train.shape

(torch.Size([941, 8]), torch.Size([941, 1]))

In [273]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(8, 1)  # One in and one out

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [274]:
input_size = 8
output_size = 1

In [275]:
model = LinearRegressionModel()

In [276]:
learning_rate = 0.1
l = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr =learning_rate)

In [277]:
num_epochs = 5000
for epoch in range(num_epochs):
    #forward feed
    y_pred = model(X_train.float().requires_grad_())

    #calculate the loss
    loss= l(y_pred.float(), y_train.float())

    #backward propagation: calculate gradients
    loss.backward()

    #update the weights
    optimizer.step()

    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    print('epoch {}, loss {}'.format(epoch, loss.item()))

Streaming output truncated to the last 5000 lines.
epoch 0, loss 1480.5396728515625
epoch 1, loss 1211.62353515625
epoch 2, loss 1050.6807861328125
epoch 3, loss 934.466552734375
epoch 4, loss 843.8687744140625
epoch 5, loss 770.2002563476562
epoch 6, loss 708.6728515625
epoch 7, loss 656.3255615234375
epoch 8, loss 611.1815795898438
epoch 9, loss 571.8458862304688
epoch 10, loss 537.2921142578125
epoch 11, loss 506.7406921386719
epoch 12, loss 479.5832824707031
epoch 13, loss 455.3355407714844
epoch 14, loss 433.6044616699219
epoch 15, loss 414.066650390625
epoch 16, loss 396.4524230957031
epoch 17, loss 380.53436279296875
epoch 18, loss 366.11907958984375
epoch 19, loss 353.04058837890625
epoch 20, loss 341.1551513671875
epoch 21, loss 330.33795166015625
epoch 22, loss 320.47967529296875
epoch 23, loss 311.484130859375
epoch 24, loss 303.2663269042969
epoch 25, loss 295.7510681152344
epoch 26, loss 288.8710632324219
epoch 27, loss 282.5664367675781
epoch 28, loss 276.78350830078125
e

In [282]:
X_train[101], y_train[101]

(tensor([0.6543, 0.2838, 0.0000, 0.3059, 0.5475, 0.1485, 0.9450, 0.4622],
        dtype=torch.float64), tensor([55.2000], dtype=torch.float64))

In [306]:
X_train[101].shape

torch.Size([8])

In [318]:
y_pred = model(test.float())
y_pred

tensor([[41.0276]], grad_fn=<AddmmBackward>)

In [221]:
scalar.fit(X_train, y_train)

tensor([34.9447], grad_fn=<AddBackward0>)

In [317]:
test = torch.from_numpy(scaler.transform(train_data.iloc[10,0:-1].to_numpy().reshape(1, -1)))
test, train_data.iloc[10,-1]

(tensor([[0.0858, 0.6121, 0.0000, 0.6436, 0.0000, 0.7151, 0.6066, 0.7479]],
        dtype=torch.float64), 39.36)